# de base de datos a json

In [5]:
from fastapi import FastAPI, requests, HTTPException
import uvicorn
import os
from dotenv import load_dotenv
import psycopg2
import json
import pandas as pd

In [6]:
load_dotenv()
    
#Leer Las Variables:
host = os.getenv("DB_HOST_AWS")
username = os.getenv("DB_USER_AWS")
password = os.getenv("DB_PASSWORD_AWS")
database = os.getenv("DB_DATABASE_AWS")
port = os.getenv("DB_PORT_AWS")


In [7]:
try:
    connection = psycopg2.connect(host=host,
                                database=database,
                                user=username,
                                password=password,
                                port=port,
                                sslmode="require")
    
    print("Conexión exitosa a la base de datos PostgreSQL con SSL")
except psycopg2.OperationalError as e:
    raise HTTPException(status_code=400, detail="Error de conexión: " + str(e))
except Exception as error:
    raise HTTPException(status_code=400, detail="Error desconocido: " + str(e))



Conexión exitosa a la base de datos PostgreSQL con SSL


In [8]:
# Escribe la consulta SQL
query = "SELECT * FROM preguntas_front"

In [ ]:
# Usa pandas para ejecutar la consulta y convertirla en un DataFrame
df = pd.read_sql_query(query, connection)
df.head(50)

In [ ]:
import json

# Agrupar opciones por pregunta y categoría
grouped = df.groupby(["id_categoria", "titulo_categoria", "texto_pregunta"])["texto_opcion"].apply(list).reset_index()

# Crear el JSON dinámico
json_data = {}
for _, row in grouped.iterrows():
    id_categoria = row["id_categoria"]
    titulo = row["titulo_categoria"]
    pregunta = row["texto_pregunta"]
    opciones = row["texto_opcion"]
    
    if id_categoria not in json_data:
        json_data[id_categoria] = {"titulo": titulo, "preguntas": {}}
    
    json_data[id_categoria]["preguntas"][pregunta] = opciones

# Convertir a JSON
json_resultado = json.dumps(json_data, indent=4, ensure_ascii=False)

print(json_resultado)

In [12]:
import json

# Agrupar opciones por pregunta y categoría
grouped = df.groupby(["id_categoria", "titulo_categoria", "texto_pregunta"])["texto_opcion"].apply(list).reset_index()

# Crear el JSON dinámico
json_data = {}
for _, row in grouped.iterrows():
    id_categoria = row["id_categoria"]
    titulo = row["titulo_categoria"]
    pregunta = row["texto_pregunta"]
    opciones = row["texto_opcion"]
    
    if id_categoria not in json_data:
        json_data[id_categoria] = {"titulo": titulo, "preguntas": {}}
    
    json_data[id_categoria]["preguntas"][pregunta] = opciones

# Convertir a JSON
json_resultado = json.dumps(json_data, indent=4, ensure_ascii=False)

print(json_resultado)


{
    "1.1": {
        "titulo": "Tengo VIH",
        "preguntas": {
            "¿Cuándo te diagnosticaron?": [
                "Hace menos de 6 meses",
                "Entre 6 meses y un año",
                "Hace más de un año"
            ],
            "¿Estás en tratamiento TAR?": [
                "Sí",
                "No",
                "No estoy seguro"
            ],
            "¿Tienes acceso a un médico?": [
                "Sí",
                "No"
            ]
        }
    },
    "1.2": {
        "titulo": "Creo que me he expuesto al virus",
        "preguntas": {
            "¿Cuándo ocurrió la posible infección?": [
                "Últimas 72h",
                "Hace más de 72h"
            ]
        }
    },
    "2.1": {
        "titulo": "Personal sanitario",
        "preguntas": {
            "¿que necesitas?": [
                "Manejo clínico de pacientes con VIH",
                "Protocolo PEP"
            ]
        }
    },
    "2.2": {
        "titulo

In [12]:
import pandas as pd

# Tabla de categorías
categorias = pd.DataFrame({
    "id_categoria": [1, 2, 3],
    "titulo_categoria": ["Tengo VIH", "Creo que me he expuesto al virus", "Personal sanitario"]
})

# Tabla de preguntas
preguntas = pd.DataFrame({
    "id_pregunta": [1, 2, 3, 4, 5],
    "id_categoria": [1, 1, 1, 2, 3],
    "texto_pregunta": [
        "¿Cuándo te diagnosticaron?",
        "¿Estás en tratamiento TAR?",
        "¿Tienes acceso a un médico?",
        "¿Cuándo ocurrió la posible infección?",
        "¿Qué necesitas?"
    ]
})

# Tabla de opciones
opciones = pd.DataFrame({
    "id_opcion": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    "id_pregunta": [1, 1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 5],
    "texto_opcion": [
        "Hace menos de 6 meses",
        "Entre 6 meses y un año",
        "Hace más de un año",
        "Sí",
        "No",
        "No estoy seguro",
        "Sí",
        "No",
        "Últimas 72h",
        "Hace más de 72h",
        "Manejo clínico de pacientes con VIH",
        "Protocolo PEP"
    ]
})



In [13]:

# 1. Unir preguntas con categorías
preguntas_categorias = pd.merge(
    preguntas,
    categorias,
    on="id_categoria",
    how="inner"
)

# 2. Unir con opciones
final_table = pd.merge(
    opciones,
    preguntas_categorias,
    on="id_pregunta",
    how="inner"
)

In [16]:
# Lista de columnas que deben ir primero
columns_first = ["id_categoria", "id_pregunta", "id_opcion"]

# Añade las columnas restantes
columns_rest = [col for col in final_table.columns if col not in columns_first]

# Reordena las columnas
final_table = final_table[columns_first + columns_rest]

print(final_table)


    id_categoria  id_pregunta  id_opcion                         texto_opcion  \
0              1            1          1                Hace menos de 6 meses   
1              1            1          2               Entre 6 meses y un año   
2              1            1          3                   Hace más de un año   
3              1            2          4                                   Sí   
4              1            2          5                                   No   
5              1            2          6                      No estoy seguro   
6              1            3          7                                   Sí   
7              1            3          8                                   No   
8              2            4          9                          Últimas 72h   
9              2            4         10                      Hace más de 72h   
10             3            5         11  Manejo clínico de pacientes con VIH   
11             3            

In [17]:
final_table

,id_categoria,id_pregunta,id_opcion,texto_opcion,texto_pregunta,titulo_categoria
0,1,1,1,Hace menos de 6 meses,¿Cuándo te diagnosticaron?,Tengo VIH
1,1,1,2,Entre 6 meses y un año,¿Cuándo te diagnosticaron?,Tengo VIH
2,1,1,3,Hace más de un año,¿Cuándo te diagnosticaron?,Tengo VIH
3,1,2,4,Sí,¿Estás en tratamiento TAR?,Tengo VIH
4,1,2,5,No,¿Estás en tratamiento TAR?,Tengo VIH
5,1,2,6,No estoy seguro,¿Estás en tratamiento TAR?,Tengo VIH
6,1,3,7,Sí,¿Tienes acceso a un médico?,Tengo VIH
7,1,3,8,No,¿Tienes acceso a un médico?,Tengo VIH
8,2,4,9,Últimas 72h,¿Cuándo ocurrió la posible infección?,Creo que me he expuesto al virus
9,2,4,10,Hace más de 72h,¿Cuándo ocurrió la posible infección?,Creo que me he expuesto al virus


In [15]:
import json

# Agrupar opciones por pregunta y categoría
grouped = df.groupby(["id_categoria", "titulo_categoria", "texto_pregunta"])["texto_opcion"].apply(list).reset_index()

# Crear el JSON dinámico
json_data = {}
for _, row in grouped.iterrows():
    id_categoria = row["id_categoria"]
    titulo = row["titulo_categoria"]
    pregunta = row["texto_pregunta"]
    opciones = row["texto_opcion"]
    
    if id_categoria not in json_data:
        json_data[id_categoria] = {"titulo": titulo, "preguntas": {}}
    
    json_data[id_categoria]["preguntas"][pregunta] = opciones

# Convertir a JSON
json_resultado = json.dumps(json_data, indent=4, ensure_ascii=False)

print(json_resultado)


{
    "1.1.1": {
        "titulo": "Tengo VIH",
        "preguntas": {
            "¿Cuándo te diagnosticaron?": [
                "Hace menos de 6 meses"
            ]
        }
    },
    "1.1.2": {
        "titulo": "Tengo VIH",
        "preguntas": {
            "¿Cuándo te diagnosticaron?": [
                "Entre 6 meses y un año"
            ]
        }
    },
    "1.1.3": {
        "titulo": "Tengo VIH",
        "preguntas": {
            "¿Cuándo te diagnosticaron?": [
                "Hace más de un año"
            ]
        }
    },
    "1.1.4": {
        "titulo": "Tengo VIH",
        "preguntas": {
            "¿Estás en tratamiento TAR?": [
                "Sí"
            ]
        }
    },
    "1.1.5": {
        "titulo": "Tengo VIH",
        "preguntas": {
            "¿Estás en tratamiento TAR?": [
                "No"
            ]
        }
    },
    "1.1.6": {
        "titulo": "Tengo VIH",
        "preguntas": {
            "¿Estás en tratamiento TAR?": [
     

# probar base de datos a json

In [ ]:


except psycopg2.OperationalError as e:
    raise HTTPException(status_code=400, detail="Error de conexión: " + str(e))
except Exception as error:
    raise HTTPException(status_code=400, detail="Error desconocido: " + str(e))
    
# Escribe la consulta SQL
query = "SELECT * FROM sociosanitarios_data"

# Usa pandas para ejecutar la consulta y convertirla en un DataFrame
df = pd.read_sql_query(query, connection)

# Agrupar opciones por pregunta y categoría
grouped = df.groupby(["id_categoria", "titulo_categoria", "texto_pregunta"])["texto_opcion"].apply(list).reset_index()

# Crear el JSON dinámico
json_data = {}
for _, row in grouped.iterrows():
    id_categoria = row["id_categoria"]
    titulo = row["titulo_categoria"]
    pregunta = row["texto_pregunta"]
    opciones = row["texto_opcion"]
    
    if id_categoria not in json_data:
        json_data[id_categoria] = {"titulo": titulo, "preguntas": {}}
    
    json_data[id_categoria]["preguntas"][pregunta] = opciones

# Convertir a JSON
json_resultado = json.dumps(json_data, indent=4, ensure_ascii=False)

print(json_resultado)
